# Deep Q-Learning on Cart Pole

For demonstration purposes: the first part of the master's project - learning of material that could match some benchmark.

Using PyTorch to train a Deep Q Learning agent on the [Cart Pole](https://gymnasium.farama.org/environments/classic_control/cart_pole/) task from [Gymnasium](https://gymnasium.farama.org/), a fork of OpenAI's Gym library to which all development of OpenAI Gym has been moved to (see https://github.com/openai/gym). This fork is a new package in the Farama Foundation, the same team of developers whom OpenAI handed over maintenance of Gym a few years ago to.

In [2]:
import gymnasium as gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

env = gym.make("CartPole-v1")  # make environment

# Set up matplotlib
if "inline" in matplotlib.get_backend():
    from IPython import display
plt.ion()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Experience Replay

Experience Replay Memory will be used for training the DQN. It stores the transitions that the agent observes, allowing the data to be reused later. By sampling from it randomly, the transitions that build up a batch are decorrelated. It has been shown that this greatly stabilises and improves the DQN training procedure.

For this, two classes are needed:
* `Transition` - a named tuple representing a single transition in the environment. It maps (state, action) pairs to their (next_state, reward) result, with the state being the screen difference image.
* `ReplayBuffer` - a cyclic buffer of bounded size that holds the transitions observed recently. It also implements a `.sample()` method for selecting a random batch of transitions for training.

In [3]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

class ReplayBuffer(object):
    """A cyclic buffer of bounded size that holds the transitions observed recently."""
    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition."""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        """Select a random batch of transitions."""
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

## DQN Algorithm

This environment is deterministic, so all equations presented here are also formulated deterministically for the sake of simplicity. In a non-deterministic environment, they would also contain expectations over stochastic transitions in the environment.

The aim is to train a policy that tries to maximize the return $G_t = r_t + \gamma r_{t+1} + \gamma^2 r_{t+2} + \cdots$, i.e. discounted sum of rewards, where discount factor $\gamma \in [0, 1]$. A small $\gamma$ implies delayed/long-term rewards are weighed less (and $\gamma = 0$ only values immediate rewards). It also encourages agents to collect reward closer in time than equivalent rewards that are temporally further away in the future. On the other hand, $\gamma = 1$ would value future rewards equally beneficial to immediate rewards 

The main idea behind Q-learning is that if we had a state-action value function $Q^*: S \times A \rightarrow \mathbb{R}$ (where $S$ and $A$ are a state and action respectively), that could provide the return if the optimal action was taken in a given state, then we could easily construct an optimal policy maximising rewards:

$$\pi^*(s) = \arg\max_a \ Q^*(s, a)$$

However, the agent doesn't know everything about the world, so won't have access to $Q^*$. However, since neural networks are universal function approximators, one can be created and trained resemble $Q^*$.

[Huber loss](https://en.wikipedia.org/wiki/Huber_loss) will be used, which acts like the mean squared error when the error is small, but like the mean absolute error when the error is large - this makes it more robust to outliers when the estimates of $Q$ are very noisy. We calculate this over a batch of transitions, $B$, sampled from the replay memory.

### Q-Network

The model will be a CNN that takes in the difference between the current and previous screen patches. It has two outputs, representing $Q(s, \mathrm{left})$ and $Q(s, \mathrm{right})$ (where $s$ is the input to the network). In effect, the network is trying to predict the expected return of taking each action given the current input.

In [4]:
class DQN(nn.Module):
    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)

    # Called with either one element to determine next action, or a batch during optimisation.
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

## Training

### Hyperparameters and Utility Functions

Some utility functions need to be defined:

* `select_action` - will select an action according to an epsilon-greedy policy. An RL agent needs to balance its actions between exploration and exploitation, so sometimes our will be used model for choosing the action (exploitation), and sometimes an action will be sampled uniformly (exploration). The probability of choosing a random action will start at `EPS_START`, decaying exponentially towards `EPS_END`. `EPS_DECAY` controls the rate of the decay.

* `plot_durations` - a helper function for plotting the durations of episodes, along with an average over the last 100 episodes (the measure used in the official evaluations). The plot will be underneath the cell containing the main training loop, and will update after every episode.

In [5]:
BATCH_SIZE = 128    # number of transitions sampled from replay buffer
GAMMA = 0.99        # discount factor
EPS_START = 0.9     # epsilon starting value
EPS_END = 0.05      # epsilon final value
EPS_DECAY = 1000    # epsilon exponential decay rate controller (higher means slower)
TAU = 0.005         # target network update rate
LR = 1e-4           # optimiser learning rate 

num_actions = env.action_space.n    # number of actions from action space
state, _ = env.reset(seed=42)
num_obs = len(state)                # number of state observations

policy_net = DQN(num_obs, num_actions).to(device)
target_net = DQN(num_obs, num_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayBuffer(10_000)

In [ ]:
steps_done = 0

def select_action(state):
    """Select an action according to an epsilon-greedy policy."""
    global steps_done
    
    eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)
    sample = random.random()
    steps_done += 1
    
    if sample > eps_threshold:
        # Use model to choose action
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        # Sample random action
        return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)


In [11]:
episode_durations = []

def plot_durations(show_result=False):
    """Plot the durations of episodes and an average over the last 100 episodes."""
    plt.figure()
    durations_t = torch.tensor(episode_durations, dtype=torch.float)

    if show_result:
        plt.title("Result")
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated

    if not show_result:
        display.display(plt.gcf())
        display.clear_output(wait=True)

### Training Loop
Here, `optimize_model` performs a single optimisation step: it samples a batch, concatenates all the tensors into one, computes $Q(s_t,a_t)$ and $V(s_{t+1}) = \max_a Q(s_{t+1}, a)$, and combines them into the loss. By definition, $V(s) = 0$ if $s$ is a terminal state. A target network is also used to compute $V(s_{t+1})$ for added stability. The target network is updated at every step with a soft update controlled by the hyperparameter `TAU` previously defined.

In [ ]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0]
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimise the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

Finally, we arrive at the main training loop. At the start, we reset the and obtain the initial `state` Tensor. Then, we sample an action, execute it, observe the next state and reward (which is always +1), and optimise the model once. When the episode ends (model fails), the loop is restarted.

In [ ]:
if torch.cuda.is_available():
    num_episodes = 600
else:
    num_episodes = 50

for i_episode in range(num_episodes):
    # Initialise the environment and get its state
    state, _ = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    
    for t in count():
        action = select_action(state)
        observation, reward, terminated, truncated, _ = env.step(action.item())
        reward = torch.tensor([reward], device=device)
        done = terminated or truncated

        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()

        # Soft update of the target network's weights
        # θ′ ← τθ + (1 − τ)θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()

        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
            
        target_net.load_state_dict(target_net_state_dict)

        if done:
            episode_durations.append(t + 1)
            plot_durations()
            break

print('Complete')
plot_durations(show_result=True)
plt.ioff()
plt.show()